Speed Detection and Car tracking

In [1]:
import sys
import os
sys.path.append(os.path.abspath("../src"))

try :
    from tracker import *
except ImportError :
    print('Import not done')

Import not done


In [2]:
!pip install ultralytics

In [3]:
import ultralytics
import cv2
import pandas as pd
from ultralytics import YOLO


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
import math


class Tracker:
    def __init__(self):
        # Store the center positions of the objects
        self.center_points = {}
        # Keep the count of the IDs
        # each time a new object id detected, the count will increase by one
        self.id_count = 0


    def update(self, objects_rect):
        # Objects boxes and ids
        objects_bbs_ids = []

        # Get center point of new object
        for rect in objects_rect:
            x, y, w, h = rect
            cx = (x + x + w) // 2
            cy = (y + y + h) // 2

            # Find out if that object was detected already
            same_object_detected = False
            for id, pt in self.center_points.items():
                dist = math.hypot(cx - pt[0], cy - pt[1])

                if dist < 35:
                    self.center_points[id] = (cx, cy)
#                    print(self.center_points)
                    objects_bbs_ids.append([x, y, w, h, id])
                    same_object_detected = True
                    break

            # New object is detected we assign the ID to that object
            if same_object_detected is False:
                self.center_points[self.id_count] = (cx, cy)
                objects_bbs_ids.append([x, y, w, h, self.id_count])
                self.id_count += 1

        # Clean the dictionary by center points to remove IDS not used anymore
        new_center_points = {}
        for obj_bb_id in objects_bbs_ids:
            _, _, _, _, object_id = obj_bb_id
            center = self.center_points[object_id]
            new_center_points[object_id] = center

        # Update dictionary with IDs not used removed
        self.center_points = new_center_points.copy()
        return objects_bbs_ids

In [6]:
model = YOLO('yolov8s.pt')

In [7]:
class_list = {'person', 'bicycle', 'car','motorcycle', 'airplane', 'bus',
              'train', 'trusk', 'traffic light'}

In [8]:
tracker = Tracker()
count = 0

In [9]:
cap = cv2.VideoCapture('/content/drive/MyDrive/Thesis pro/highway.mp4')

In [10]:
from google.colab.patches import cv2_imshow
while True:
    ret, frame = cap.read()
    if not ret:
        break
    count += 1
    frame = cv2.resize(frame,(1020,500))

    result = model.predict(frame)
    # print(result)
    a = result[0].boxes.data
    a = a.detach().cpu().numpy() # added this line
    px = pd.DataFrame(a).astype("float")
    # print(px)

    list = []

    for index, row in px.iterrows():
      print(row)
      x1 = int(row[0])
      x2 = int(row[2])
      y1 = int(row[1])
      y2 = int(row[3])
      d = int(row[5])
      c = class_list[d]
      if 'car' in c:
        list.append([x1,y1,x2,y2])
        # print(c)


    cv2_imshow(frame)
    if cv2.waitKey(0) & 0xFF == 27:
        break
cap.release()
cv2.destroyAllWindows()


0: 320x640 8 cars, 2 trucks, 805.0ms
Speed: 3.6ms preprocess, 805.0ms inference, 9.3ms postprocess per image at shape (1, 3, 320, 640)
0      0.146739
1    271.796753
2     66.228218
3    329.035400
4      0.855663
5      2.000000
Name: 0, dtype: float64


TypeError: 'set' object is not subscriptable